In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
# import necessary packages

import os
import numpy as np
import pandas as pd
import cv2
import shutil
import ast

import seaborn as sns
from matplotlib import pyplot as plt

from collections import defaultdict
from scipy.signal import find_peaks
import scipy

from sklearn.feature_selection import f_classif
from extract_feature_utils import *
from stat_vis_utils  import *
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score, cohen_kappa_score, accuracy_score

In [ ]:
DIR_WSI = "/Users/jinzhou/Desktop/Paper_MIA/data/wsi"
DIR_ANN = "/Users/jinzhou/Desktop/Paper_MIA/data/ann_geojsons"
DIR_SAVE_RESULTS = "/Users/jinzhou/Desktop/Paper_MIA/results"
DIR_SAVE_FIGURE = "/Users/jinzhou/Desktop/Paper_MIA/results/figures"

In [ ]:
# From Jeff

PATH_LABEL = "/Users/jinzhou/Desktop/Paper_MIA/data/Batch 1,2,3 Art Scores COMPLETE.xlsx"
df_label = pd.ExcelFile(PATH_LABEL)
df_label_batch_1 = df_label.parse(df_label.sheet_names[0], skiprows=1)
df_label_batch_1 = df_label_batch_1.rename(columns = {'Unnamed: 0':'Artery_ID'}).set_index("Artery_ID")
df_label_batch_2 = df_label.parse(df_label.sheet_names[1], skiprows=1)
df_label_batch_2 = df_label_batch_2.rename(columns = {'Unnamed: 0':'Artery_ID'}).set_index("Artery_ID")
df_label_batch_3 = df_label.parse(df_label.sheet_names[2], skiprows=1)
df_label_batch_3 = df_label_batch_3.rename(columns = {'Unnamed: 0':'Artery_ID'}).set_index("Artery_ID")
df_label_jeff = pd.concat([df_label_batch_1, df_label_batch_2, df_label_batch_3], axis=1)
df_label_jeff = df_label_jeff.replace({"without arteriosclerosis": 0, "mild arteriosclerosis": 1, 
                                       "mild hyalinosis": 1, "moderate arteriosclerosis": 2, 
                                       "severe arteriosclerosis": 3, 
                                       "0 (no green line - reject)": np.nan, "1 (no green line - reject)": np.nan,
                                       "reject, no lumen": np.nan, "fix": np.nan, "-": np.nan, " - ": np.nan})

In [ ]:
df_label_jeff.to_csv("/Users/jinzhou/Desktop/Paper_MIA/data/labels_jeff.csv")

In [ ]:
# From David
PATH_LABEL_2 = "/Users/jinzhou/Desktop/Paper_MIA/data/Batch 2 Art Scores from David_Completed.xlsx"
PATH_LABEL_3 = "/Users/jinzhou/Desktop/Paper_MIA/data/Batch 3 Art Scores from David_Completed.xlsx"
df_label_batch_2 = pd.ExcelFile(PATH_LABEL_2)
df_label_batch_2 = df_label_batch_2.parse(df_label_batch_2.sheet_names[0], skiprows=1)
df_label_batch_2 = df_label_batch_2.rename(columns = {'Unnamed: 0':'Artery_ID'}).set_index("Artery_ID")

df_label_batch_3 = pd.ExcelFile(PATH_LABEL_3)
df_label_batch_3 = df_label_batch_3.parse(df_label_batch_3.sheet_names[0], skiprows=1)
df_label_batch_3 = df_label_batch_3.rename(columns = {'Unnamed: 0':'Artery_ID'}).set_index("Artery_ID")

df_label_david = pd.concat([df_label_batch_2, df_label_batch_3], axis=1)
df_label_david = df_label_david.replace({"without arteriosclerosis": 0, "mild arteriosclerosis": 1, 
                                       "mild hyalinosis": 1, "Mild arteriosclerosis": 1,
                                        "moderate arteriosclerosis": 2, 
                                         
                                       "severe arteriosclerosis": 3, 
                                       "0 (no green line - reject)": np.nan, "1 (no green line - reject)": np.nan,
                                       "reject, no lumen": np.nan, "fix": np.nan, "-": np.nan, " - ": np.nan})

In [ ]:
df_label_david.to_csv("/Users/jinzhou/Desktop/Paper_MIA/data/labels_david.csv")

In [ ]:
# df_label_david = df_label_david.loc[:, ['015_TRI_235805',  '2_4827_A_0045173', '2_4839_A_0039862',
#        '2_4841_A_0045207', '2_6063_A_0045151', '2_6094_A_0039851',
#        '2_6097_A_0045196', '2_6128_A_0037277', '2_6130_A_0037288']]

In [ ]:
wsi_ids_common = df_label_david.columns

In [ ]:
wsi_ids_common

In [ ]:
for col in wsi_ids_common:
    
    df_jeff_col = df_label_jeff.loc[:, [col]]
    df_david_col = df_label_david.loc[:, [col]]
    
    stacked_jeff = df_jeff_col.stack()
    stacked_jeff.index = stacked_jeff.index.map('{0[1]}_{0[0]}'.format)
    df_jeff_col = stacked_jeff.to_frame()
    df_jeff_col.columns = ["Jeff"]
    
    stacked_david = df_david_col.stack()
    stacked_david.index = stacked_david.index.map('{0[1]}_{0[0]}'.format)
    df_david_col = stacked_david.to_frame()
    df_david_col.columns = ["David"]
    
    df_label = pd.concat([df_jeff_col, df_david_col], axis=1)
    df_label = df_label.dropna()
    y_jeff = df_label.loc[:, "Jeff"].values
    y_david = df_label.loc[:, "David"].values
    
    print(col, cohen_kappa_score(y_jeff, y_david))
    print(confusion_matrix(y_jeff, y_david))
    
    

In [ ]:
wsi_ids_test = [ '16-032-02_TRI_11628_copy', '2_4827_A_0045173', 
       '2_4841_A_0045207', '2_6094_A_0039851',
       '2_6097_A_0045196', '2_6128_A_0037277', '2_6130_A_0037288']


In [ ]:
df_label_david = df_label_david.loc[:, wsi_ids_test]

In [ ]:
stacked_jeff = df_label_jeff.stack()
stacked_jeff.index = stacked_jeff.index.map('{0[1]}_{0[0]}'.format)
df_label_jeff = stacked_jeff.to_frame()
df_label_jeff.columns = ["Jeff"]
df_label_jeff

In [ ]:
stacked_david = df_label_david.stack()
stacked_david.index = stacked_david.index.map('{0[1]}_{0[0]}'.format)
df_label_david = stacked_david.to_frame()
df_label_david.columns = ["David"]
df_label_david

In [ ]:
df_label = pd.concat([df_label_jeff, df_label_david], axis=1)

In [ ]:
df_label = df_label.dropna()

In [ ]:
y_jeff = df_label.loc[:, "Jeff"].values
y_david = df_label.loc[:, "David"].values

In [ ]:
confusion_matrix(y_jeff, y_david)

In [ ]:
cohen_kappa_score(y_jeff, y_david)

In [ ]:
scipy.stats.kendalltau(y_jeff, y_david)

In [ ]:
y_jeff[y_jeff>=1]=1
y_david[y_david>=1]=1
cohen_kappa_score(y_jeff, y_david)

In [ ]:
confusion_matrix(y_jeff, y_david)